In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import ensemble
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [2]:
Base = pd.read_csv('C:/Users/fabiopda/Desktop/MBA/Projeto Final/Original.csv')
Base = Base.drop(['Unnamed: 0'], axis=1)

## Criação de novas variáveis - Apostas

In [3]:
resultado = []
for linha in range(0,len(Base)):
    if Base['B365H'][linha] == Base['B365A'][linha]:
        resultado.append(0)
    else:
        resultado.append(Base['B365H'][linha]/Base['B365A'][linha])
Base['B365F'] = resultado

resultado = []
for linha in range(0,len(Base)):
    if Base['BWH'][linha] == Base['BWA'][linha]:
        resultado.append(0)
    else:
        resultado.append(Base['BWH'][linha]/Base['BWA'][linha])
Base['BWF'] = resultado

resultado = []
for linha in range(0,len(Base)):
    if Base['IWH'][linha] == Base['IWA'][linha]:
        resultado.append(0)
    else:
        resultado.append(Base['IWH'][linha]/Base['IWA'][linha])
Base['IWF'] = resultado

resultado = []
for linha in range(0,len(Base)):
    if Base['LBH'][linha] == Base['LBA'][linha]:
        resultado.append(0)
    else:
        resultado.append(Base['LBH'][linha]/Base['LBA'][linha])
Base['LBF'] = resultado

resultado = []
for linha in range(0,len(Base)):
    if Base['WHH'][linha] == Base['WHA'][linha]:
        resultado.append(0)
    else:
        resultado.append(Base['WHH'][linha]/Base['WHA'][linha])
Base['WHF'] = resultado

resultado = []
for linha in range(0,len(Base)):
    if Base['VCH'][linha] == Base['VCA'][linha]:
        resultado.append(0)
    else:
        resultado.append(Base['VCH'][linha]/Base['VCA'][linha])
Base['VCF'] = resultado

## Criação da variável resposta

In [4]:
resultado = []

for linha in range(0,len(Base)):
    if Base['home_team_goal'][linha] > Base['away_team_goal'][linha]:
        resultado.append(1)
    elif Base['home_team_goal'][linha] == Base['away_team_goal'][linha]:
        resultado.append(0)
    else:
        resultado.append(0)
Base['Resultado_Casa'] = resultado

Base = Base.drop(['home_team_goal','away_team_goal'], axis=1)

## Separação da base de modelagem e de aplicação

In [5]:
Base_Aplicacao = Base.query("Ano_2016 > 0")
Base = Base.query("Ano_2016 == 0")
Base = Base.drop(['Ano_2009','Ano_2010','Ano_2011','Ano_2012','Ano_2013','Ano_2014','Ano_2015','Ano_2016'], axis=1)

## Separação da base de treino e teste

In [6]:
X = Base.drop(['Resultado_Casa'], axis=1)
y = Base.Resultado_Casa

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit only to the training data
scaler.fit(X_train)
# Now apply the transformations to the data:
X_train2 = scaler.transform(X_train)
X_test2 = scaler.transform(X_test)

## Logistic Regression

In [8]:
tuned_parameters = [{'C': [0.0001,0.001,0.002],'max_iter':[15,25,50],'random_state':[42]}]
clf = GridSearchCV(LogisticRegression(), tuned_parameters, cv=3, scoring='accuracy')
clf.fit(X_train2, y_train)

#tuned_parameters = [{'C': [0.01,0.1,1],'max_iter':[100,200,1000],'random_state':[42]}]
#{'C': 0.01, 'max_iter': 100, 'random_state': 42}
#Accuracy: 0.5741
#AUC: 0.5000

#tuned_parameters = [{'C': [0.001,0.01,0.05],'max_iter':[50,100,150],'random_state':[42]}]
#{'C': 0.001, 'max_iter': 50, 'random_state': 42}
#Accuracy: 0.6000
#AUC: 0.5915

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'C': [0.0001, 0.001, 0.002], 'max_iter': [15, 25, 50], 'random_state': [42]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [9]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, clf.predict(X_test2))
print("Accuracy: %.4f" % acc)
print()

probs = clf.predict_proba(X_test)
auc_ = roc_auc_score(y_test, probs[:,1])
print("AUC: %.4f" % auc_)

Best parameters set found on development set:

{'C': 0.0001, 'max_iter': 15, 'random_state': 42}

Grid scores on development set:

0.624 (+/-0.011) for {'C': 0.0001, 'max_iter': 15, 'random_state': 42}
0.624 (+/-0.011) for {'C': 0.0001, 'max_iter': 25, 'random_state': 42}
0.624 (+/-0.011) for {'C': 0.0001, 'max_iter': 50, 'random_state': 42}
0.623 (+/-0.025) for {'C': 0.001, 'max_iter': 15, 'random_state': 42}
0.623 (+/-0.025) for {'C': 0.001, 'max_iter': 25, 'random_state': 42}
0.623 (+/-0.025) for {'C': 0.001, 'max_iter': 50, 'random_state': 42}
0.616 (+/-0.009) for {'C': 0.002, 'max_iter': 15, 'random_state': 42}
0.616 (+/-0.009) for {'C': 0.002, 'max_iter': 25, 'random_state': 42}
0.616 (+/-0.009) for {'C': 0.002, 'max_iter': 50, 'random_state': 42}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Accuracy: 0.6112

AUC: 0.6259


## Decision Tree

In [10]:
tuned_parameters = [{'max_depth': [2,3,7,10],
                     'min_samples_split': [2,3,5,10],
                     'criterion': ['gini','entropy'],
                     'random_state':[42]}]

clf = GridSearchCV(DecisionTreeClassifier(), tuned_parameters, cv=3, scoring='accuracy')
clf.fit(X_train, y_train)

#tuned_parameters = [{'max_depth': [2,3,5],'min_samples_split': [1,2,3,4,7],'criterion': ['gini','entropy'],'random_state':[42]}]
#{'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 2, 'random_state': 42}
#Accuracy: 0.6231
#AUC: 0.6559

#tuned_parameters = [{'max_depth': [2,3,7,10],'min_samples_split': [2,3,5,10],'criterion': ['gini','entropy'],'random_state':[42]}]
#{'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 2, 'random_state': 42}
#Accuracy: 0.6231
#AUC: 0.6559

GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'max_depth': [2, 3, 7, 10], 'min_samples_split': [2, 3, 5, 10], 'criterion': ['gini', 'entropy'], 'random_state': [42]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [11]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, clf.predict(X_test))
print("Accuracy: %.4f" % acc)
print()

probs = clf.predict_proba(X_test)
auc_ = roc_auc_score(y_test, probs[:,1])
print("AUC: %.4f" % auc_)

Best parameters set found on development set:

{'criterion': 'entropy', 'max_depth': 2, 'min_samples_split': 2, 'random_state': 42}

Grid scores on development set:

0.626 (+/-0.015) for {'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 2, 'random_state': 42}
0.626 (+/-0.015) for {'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 3, 'random_state': 42}
0.626 (+/-0.015) for {'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 5, 'random_state': 42}
0.626 (+/-0.015) for {'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 10, 'random_state': 42}
0.606 (+/-0.005) for {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2, 'random_state': 42}
0.606 (+/-0.005) for {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 3, 'random_state': 42}
0.606 (+/-0.005) for {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 5, 'random_state': 42}
0.606 (+/-0.005) for {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 10, 'random_state': 42}
0.580 (+

## Random Forest

In [12]:
tuned_parameters = [{'n_estimators': [100,200,225],
                     'max_depth': [15,17,19],
                     'criterion':['gini','entropy'],
                     'random_state':[42],
                     'max_features': [45,50,55]}]

clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=3, scoring='accuracy')
clf.fit(X_train, y_train)

#tuned_parameters = [{'n_estimators': [100,250,300],
#                     'max_depth': [1,3,5,15,20,25,50],
#                     'criterion':['gini','entropy'],
#                     'random_state':[42],
#                     'max_features': [25,50,100,1146]}]

#{'criterion': 'gini', 'max_depth': 20, 'max_features': 50, 'n_estimators': 250, 'random_state': 42}
#Accuracy: 0.6027
#AUC: 0.6443

#tuned_parameters = [{'n_estimators': [200,250,275],
#                     'max_depth': [19,20,21],
#                     'criterion':['gini','entropy'],
#                     'random_state':[42],
#                     'max_features': [40,50,60]}]

#{'criterion': 'gini', 'max_depth': 19, 'max_features': 50, 'n_estimators': 200, 'random_state': 42}
#Accuracy: 0.5891
#AUC: 0.6422

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [100, 200, 225], 'max_depth': [15, 17, 19], 'criterion': ['gini', 'entropy'], 'random_state': [42], 'max_features': [45, 50, 55]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [13]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, clf.predict(X_test))
print("Accuracy: %.4f" % acc)
print()

probs = clf.predict_proba(X_test)
auc_ = roc_auc_score(y_test, probs[:,1])
print("AUC: %.4f" % auc_)

Best parameters set found on development set:

{'criterion': 'gini', 'max_depth': 15, 'max_features': 55, 'n_estimators': 200, 'random_state': 42}

Grid scores on development set:

0.642 (+/-0.017) for {'criterion': 'gini', 'max_depth': 15, 'max_features': 45, 'n_estimators': 100, 'random_state': 42}
0.637 (+/-0.031) for {'criterion': 'gini', 'max_depth': 15, 'max_features': 45, 'n_estimators': 200, 'random_state': 42}
0.640 (+/-0.031) for {'criterion': 'gini', 'max_depth': 15, 'max_features': 45, 'n_estimators': 225, 'random_state': 42}
0.632 (+/-0.015) for {'criterion': 'gini', 'max_depth': 15, 'max_features': 50, 'n_estimators': 100, 'random_state': 42}
0.642 (+/-0.028) for {'criterion': 'gini', 'max_depth': 15, 'max_features': 50, 'n_estimators': 200, 'random_state': 42}
0.635 (+/-0.034) for {'criterion': 'gini', 'max_depth': 15, 'max_features': 50, 'n_estimators': 225, 'random_state': 42}
0.638 (+/-0.012) for {'criterion': 'gini', 'max_depth': 15, 'max_features': 55, 'n_estimators

## GRADIENT BOOSTING

In [ ]:
tuned_parameters = [{'n_estimators': [10, 100],
                     'max_depth' : [4, 5, 6],
                     'min_samples_split': [2, 5, 10],
                     'criterion':['mae'],
                     'random_state':[42],
                     'learning_rate': [0.03, 0.05], 
                    # 'subsample': [0.5, 1]
                    }]
clf = GridSearchCV(ensemble.GradientBoostingClassifier(), tuned_parameters, cv=3)
clf.fit(X_train, y_train)

In [ ]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, clf.predict(X_test)))
print()
predictions = clf.predict(X_test)
acc = roc_auc_score(y_test, predictions)
print("accuracy_score: %.4f" % acc)

probs = clf.predict_proba(X_test)
auc_ = roc_auc_score(y_test, probs[:,1])
print("AUC: %.4f" % auc_)

# SVC

In [ ]:
tuned_parameters = [{'kernel': ['rbf', 'linear', 'poly'],
                     'C': [1,10,20]}]
clf = GridSearchCV(SVC(), tuned_parameters, cv=3)
clf.fit(X_train2, y_train)

In [ ]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, clf.predict(X_test)))
print()

predictions = clf.predict(X_test)
acc = roc_auc_score(y_test, predictions)
print("accuracy_score: %.4f" % acc)

probs = clf.predict_proba(X_test)
auc_ = roc_auc_score(y_test, probs[:,1])
print("AUC: %.4f" % auc_)

## RNA

In [ ]:
tuned_parameters = [{'hidden_layer_sizes': [(1,),(50,50),(50,50,50),(50,50,50,50,50),(50,150,50,100,50)],
                    'activation' : ['logistic', 'relu','tanh'],
                    'learning_rate': ['constant', 'adaptive'],
                    'alpha': [0.001,0.01,0.1],
                    'random_state' : [42],
                    'max_iter': [2000,1000],
                    'solver': ['sgd','adam','lbfgs']}]
clf = GridSearchCV(MLPClassifier(), tuned_parameters, cv=4)
clf.fit(X_train2, y_train)

In [ ]:
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
print(classification_report(y_test, clf.predict(X_test2)))
print()

predictions = clf.predict(X_test)
acc = roc_auc_score(y_test, predictions)
print("accuracy_score: %.4f" % acc)

probs = clf.predict_proba(X_test2)
auc_ = roc_auc_score(y_test, probs[:,1])
print("AUC: %.4f" % auc_)


In [ ]:
probs = clf.predict_proba(X_test2)
auc_ = roc_auc_score(y_test, probs[:,1])
print("AUC: %.4f" % auc_)
#false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, clf.predict_proba(xtest)[:,1])
#print(auc(false_positive_rate, true_positive_rate),roc_auc_score(Y_test, clf.predict_proba(xtest)))